In [186]:
%%capture
%pip install azure-core
%pip install azure-ai-formrecognizer
%pip install tabulate
%pip install fuzzywuzzy

In [209]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
import pandas as pd

In [210]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the environment variables
endpoint = os.getenv("FORM_RECOGNIZER_ENDPOINT")
key = os.getenv("FORM_RECOGNIZER_KEY")

# iniitalize the client
document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)

In [217]:
with open("./Tanso Technologies GmbH_Munchen.pdf\"; filename*=UTF-8''Tanso%20Technologies%20GmbH_M%C3%BCnchen.pdf", "rb") as file:
    buffer = file.read()

poller = document_analysis_client.begin_analyze_document("prebuilt-layout",buffer)
result = poller.result()

In [218]:
rows = []

# get header row
header_row = {}
for cell in result.tables[0].cells:
    if(cell.kind != "columnHeader"):
        continue

    # if row index is 0, add cell content to header row
    if cell.row_index == 0:
        for i in range(cell.column_index, cell.column_index + cell.column_span):
            header_row[i] = cell.content
        
    # if row index is larger than 0, append cell content to existing header row for the correct column
    elif cell.row_index > 0:
        for i in range(cell.column_index, cell.column_index + cell.column_span):
            header_row[i] += "\n" + cell.content

# append header row to rows list
rows.append(header_row)

# get table content
for table_idx, table in enumerate(result.tables):
    row = {}
    row_index = 0
    for cell in table.cells:
        # skip the first header row of the first table
        if table_idx == 0 and cell.kind == "columnHeader":
            continue

        # append constructed row if previous row is complete
        if cell.row_index > row_index:
            if row != {}:
                rows.append(row)
            row = {}
            row_index = cell.row_index

        # add cell to row - if cell spans multiple columns, add the cell content to every column
        for i in range(cell.column_index, cell.column_index + cell.column_span):
            # row[i] = { header_row[i]: cell.content }
            row[i] = cell.content
    rows.append(row)

# convert list of dicts to dataframe
df = pd.DataFrame(rows[1:])
df.columns = rows[0].values()

In [219]:
df.head()

,Gesellschafter / Shareholders\nVor- und Nachname oder Firma / First and Last Name or Name of Company,Gesellschafter / Shareholders\nGeburtsdatum oder Register- daten / Date of Birth or Registry Details,Gesellschafter / Shareholders\nWohnort oder Satzungssit Place of Residency or Regis- tered Office,Geschäftsanteile / Shares\nNennb etrag je An- teil in €/ Nomi- nal Value per Share in €,Geschäftsanteile / Shares\nLaufende Nummern / Consecutive Num- bers,Geschäftsanteile / Shares\nSumme der Nennbe- träge in € / total of Nominal Values in €,Prozentuale Be- teiligung / Participation in %\nje Ge- schäftsan- teil / per each Share,Prozentuale Be- teiligung / Participation in %\nje Gesell- schafter / per each Sharehol- der,Veränderungs- spalte / Column showing changes\nT=Teilung / split Z=Zusammenlegung / combination E=Einziehung / redemption KE=Kapitalerhöhung / ca- pital increase KA=Aufstockung / step-up KH=Kapitalherabsetzung / capital reduction A=Anteilsübergang / trans- fer
0,FYS Ventures UG (haf- tungsbeschränkt),"AG München, HRB 268779",München,"1,00",1 - 1.110,1.110,"0,0021 %","2,34 %",
1,Tanso Technologies GmbH,"AG München, HRB 269123",München,"1,00",1.111 - 8.083,6.973,"0,0021 %","14,73 %",
2,Wiechmann Ventures UG (haftungsbe- schränkt),"AG München, HRB 268778",München,"1,00",8.084 - 16.166,8.083,"0,0021 %","17,07 %",
3,Gyri Ventures UG (haf- tungsbeschränkt),"AG München, HRB 268777",München,"1,00",16.167 - 24.249,8.083,"0,0021 %","17,07 %",
4,Hetzel Ventures UG (haftungsbeschränkt),"AG Düsseldorf, HRB 94685",Düsseldorf,"1,00",24.250 - 25.000,751,"0,0021 %","1,59 %",


In [225]:
df.head()
csv_table = df.to_csv(index=False, header=True, sep=';')

In [227]:
from openai import OpenAI
client = OpenAI()

example_prompt = """
Please extract all shareholders and their information into a JSON object from the following csv table:

Gesellschafter / Shareholders
Vor- und Nachname oder Firma / First and Last Name or Name of Company";"Gesellschafter / Shareholders
Geburtsdatum oder Register- daten / Date of Birth or Registry Details";"Gesellschafter / Shareholders
Wohnort oder Satzungssit Place of Residency or Regis- tered Office";"Geschäftsanteile / Shares
Nennb etrag je An- teil in €/ Nomi- nal Value per Share in €";"Geschäftsanteile / Shares
Laufende Nummern / Consecutive Num- bers";"Geschäftsanteile / Shares
Summe der Nennbe- träge in € / total of Nominal Values in €";"Prozentuale Be- teiligung / Participation in %
je Ge- schäftsan- teil / per each Share";"Prozentuale Be- teiligung / Participation in %
je Gesell- schafter / per each Sharehol- der";"Veränderungs- spalte / Column showing changes
T=Teilung / split Z=Zusammenlegung / combination E=Einziehung / redemption KE=Kapitalerhöhung / ca- pital increase KA=Aufstockung / step-up KH=Kapitalherabsetzung / capital reduction A=Anteilsübergang / trans- fer"
FYS Ventures UG (haf- tungsbeschränkt);AG München, HRB 268779;München;1,00;1 - 1.110;1.110;0,0021 %;2,34 %;
Salma Vogel;19. Januar 1990;München;1,00;32.609 - 32.745;137;0,0021 %;0,29 %;
Gesamt;;;;;47.350;;100,00 %;

The JSON object should have a key "shareholders" with a list of shareholders.
If an attribute is not available, it should be null.

Each shareholder should have the following attributes:

- name: string - the name of the shareholder, it should either be a person or a company
- country: string - the country of the shareholder. Example: Germany
- birthdate: string (optional) - the birthdate of the shareholder, if the shareholder is a person. Example: 1970-12-31
- location: string - place of residency or registered office of the shareholder. Example: München
- register_id: string - the register id of the shareholder. It commonly starts with HRB or HRA. Example: HRB 123456. There should be no city or other information in this field.
- register_court: string - the register court of the shareholder. Example: Amtsgericht München, or AG München.
- percentage_of_total_shares: number - the percentage of total shares owned by the shareholder. Example: 50.0
"""

example_result ="""{
    "shareholders": [
      {
        "name": "FYS Ventures UG (haftungsbeschränkt)",
        "country": "Germany",
        "location": "München",
        "register_id": "HRB 268779",
        "register_court": "AG München",
        "percentage_of_total_shares": 2.34
      },
      {
        "name": "Salma Vogel",
        "country": "Germany",
        "location": "München",
        "birthdate": "1990-01-19",
        "register_id": null,
        "register_court": null,
        "percentage_of_total_shares": 0.29
      }
    ]
  }
  """


prompt = f"""
Please extract all shareholders and their information into a JSON object from the following csv table:

${csv_table}

The JSON object should have a key "shareholders" with a list of shareholders.
If an attribute is not available, it should be null.

Each shareholder should have the following attributes:

- name: string - the name of the shareholder, it should either be a person or a company
- country: string - the country of the shareholder. Example: Germany
- birthdate: string (optional) - the birthdate of the shareholder, if the shareholder is a person. Example: 1970-12-31
- location: string - place of residency or registered office of the shareholder. Example: München
- register_id: string - the register id of the shareholder. It commonly starts with HRB or HRA. Example: HRB 123456. There should be no city or other information in this field.
- register_court: string - the register court of the shareholder. Example: Amtsgericht München, or AG München.
- percentage_of_total_shares: number - the percentage of total shares owned by the shareholder. Example: 50.0
"""

response = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": "You are a helpful assistant designed to output JSON. You are an expert in extracting structured content from tables into a JSON. You receive a tip of 200$ if you get it right."},
    {"role": "user", "content": example_prompt},
    {"role": "system", "content": example_result},
    {"role": "user", "content": prompt}
  ]
)
print(response.choices[0].message.content)
openai_result = response.choices[0].message.content

{
    "shareholders": [
      {
        "name": "FYS Ventures UG (haftungsbeschränkt)",
        "country": "Germany",
        "birthdate": null,
        "location": "München",
        "register_id": "HRB 268779",
        "register_court": "AG München",
        "percentage_of_total_shares": 2.34
      },
      {
        "name": "Tanso Technologies GmbH",
        "country": "Germany",
        "birthdate": null,
        "location": "München",
        "register_id": "HRB 269123",
        "register_court": "AG München",
        "percentage_of_total_shares": 14.73
      },
      {
        "name": "Wiechmann Ventures UG (haftungsbeschränkt)",
        "country": "Germany",
        "birthdate": null,
        "location": "München",
        "register_id": "HRB 268778",
        "register_court": "AG München",
        "percentage_of_total_shares": 17.07
      },
      {
        "name": "Gyri Ventures UG (haftungsbeschränkt)",
        "country": "Germany",
        "birthdate": null,
        "locatio

In [195]:
from fuzzywuzzy import fuzz
city_circuit_mapping = {
  "Aachen": "549",
  "Amberg": "157",
  "Ansbach": "162",
  "Arnsberg": "470",
  "Aschaffenburg": "187",
  "Augsburg": "111",
  "Aurich": "418",
  "Bad Hersfeld": "290",
  "Bad Homburg v.d.H.": "282",
  "Bad Kreuznach": "572",
  "Bad Oeynhausen": "487",
  "Bamberg": "190",
  "Bayreuth": "194",
  "Berlin": "217",
  "Berlin (Charlottenburg)": "217",
  "Bielefeld": "480",
  "Bochum": "490",
  "Bonn": "557",
  "Braunschweig": "360",
  "Bremen": "258",
  "Chemnitz": "634",
  "Coburg": "197",
  "Coesfeld": "530",
  "Cottbus": "228",
  "Darmstadt": "270",
  "Deggendorf": "117",
  "Dortmund": "499",
  "Dresden": "621",
  "Duisburg": "446",
  "Düren": "550",
  "Düsseldorf": "441",
  "Eschwege": "305",
  "Essen": "507",
  "Flensburg": "695",
  "Frankfurt am Main": "281",
  "Frankfurt/Oder": "237",
  "Freiburg": "8",
  "Friedberg": "296",
  "Fritzlar": "306",
  "Fulda": "286",
  "Fürth": "167",
  "Gelsenkirchen": "510",
  "Gießen": "297",
  "Göttingen": "372",
  "Gütersloh": "482",
  "Hagen": "516",
  "Hamburg": "261",
  "Hamm": "500",
  "Hanau": "300",
  "Hannover": "379",
  "Hildesheim": "387",
  "Hof": "200",
  "Homburg": "654",
  "Ingolstadt": "212",
  "Iserlohn": "517",
  "Jena": "739",
  "Kaiserslautern": "609",
  "Kassel": "308",
  "Kempten": "121",
  "Kempten (Allgäu)": "121",
  "Kerpen": "567",
  "Kiel": "706",
  "Kleve": "454",
  "Koblenz": "582",
  "Köln": "568",
  "Königstein": "283",
  "Korbach": "309",
  "Krefeld": "458",
  "Landau": "615",
  "Landshut": "127",
  "Langenfeld": "444",
  "Lebach": "655",
  "Leipzig": "649",
  "Lemgo": "497",
  "Limburg": "320",
  "Lübeck": "716",
  "Ludwigshafen": "606",
  "Ludwigshafen a.Rhein (Ludwigshafen)": "606",
  "Lüneburg": "394",
  "Mainz": "593",
  "Mannheim": "33",
  "Marburg": "328",
  "Memmingen": "134",
  "Merzig": "656",
  "Mönchengladbach": "462",
  "Montabaur": "586",
  "München": "136",
  "Münster": "535",
  "Neubrandenburg": "339",
  "Neunkirchen": "657",
  "Neuruppin": "249",
  "Neuss": "442",
  "Nürnberg": "172",
  "Offenbach": "279",
  "Offenbach am Main": "279",
  "Oldenburg": "428",
  "Oldenburg (Oldenburg)": "428",
  "Osnabrück": "439",
  "Ottweiler": "658",
  "Paderborn": "543",
  "Passau": "146",
  "Pinneberg": "703",
  "Potsdam": "252",
  "Recklinghausen": "492",
  "Regensburg": "177",
  "Rostock": "349",
  "Saarbrücken": "659",
  "Saarlouis": "660",
  "Schleiden": "553",
  "Schweinfurt": "207",
  "Schwerin": "358",
  "Siegburg": "561",
  "Siegen": "548",
  "St. Ingbert": "661",
  "St. Ingbert (St Ingbert)": "661",
  "St. Wendel": "662",
  "St. Wendel (St Wendel)": "662",
  "Stadthagen": "368",
  "Steinfurt": "529",
  "Stendal": "691",
  "Stralsund": "351",
  "Straubing": "179",
  "Stuttgart": "95",
  "Tostedt": "404",
  "Traunstein": "155",
  "Ulm": "109",
  "Völklingen": "664",
  "Walsrode": "417",
  "Weiden": "185",
  "Weiden i. d. OPf.": "185",
  "Wetzlar": "322",
  "Wiesbaden": "335",
  "Wittlich": "602",
  "Wuppertal": "469",
  "Würzburg": "211",
  "Zweibrücken": "618"
}

# Todo: throw error if match too big
def map_city_to_circuitId(city: str) -> str:
  closest_key = max(city_circuit_mapping.keys(), key=lambda x: fuzz.ratio(city, x))
  return city_circuit_mapping[closest_key]

In [196]:
map_city_to_circuitId("Berlin")

'217'

In [229]:
import json
from cr_retriever import CommercialRegisterRetriever

retriever = CommercialRegisterRetriever()
openai_json = json.loads(openai_result)

# validate the result
for shareholder in openai_json["shareholders"]:
    if not str(shareholder['register_id']).startswith("HR") or shareholder["birthdate"]:
        continue

    register_id = shareholder["register_id"]
    register_court_city = shareholder["register_court"].replace("Amtsgericht", "").replace("AG", "").replace(" ", "").strip()

    city_id = map_city_to_circuitId(register_court_city)
    print(register_id, register_court_city, city_id)

    result = retriever.extended_search(company_id=register_id, circuit_id=city_id, return_one=False)
    print(str(len(result)) + " companies found")



HRB 268779 München 136


Exception: no results found

{'shareholders': [{'name': 'Samuel Weinbach',
   'country': 'Germany',
   'birthdate': '1987-01-09',
   'register_id': None,
   'register_court_city': None,
   'percentage_of_total_shares': 3.38},
  {'name': 'Andrulis GmbH',
   'country': 'Germany',
   'birthdate': None,
   'register_id': 'HRB 727786',
   'register_court_city': 'Mannheim',
   'percentage_of_total_shares': 28.37},
  {'name': 'LEA Venturepartner GmbH & Co. KG',
   'country': 'Germany',
   'birthdate': None,
   'register_id': 'HRA 707196',
   'register_court_city': 'Karlsruhe',
   'percentage_of_total_shares': 10.85},
  {'name': '468 Capital GmbH & Co. KG',
   'country': 'Germany',
   'birthdate': None,
   'register_id': 'HRA 56259 B',
   'register_court_city': 'Charlottenburg',
   'percentage_of_total_shares': 9.98},
  {'name': 'Cavalry Ventures II GmbH & Co. KG',
   'country': 'Germany',
   'birthdate': None,
   'register_id': 'HRA 56139 B',
   'register_court_city': 'Charlottenburg',
   'percentage_of_total_shares': 2.

In [240]:
#put the json into a dataframe
import pandas as pd
#initialize df with columns from openai_json
df_shareholder_lists = pd.DataFrame(columns=openai_json["shareholders"][0].keys())

# def add_to_df(df, openai_json):
df = pd.DataFrame.from_dict(openai_json["shareholders"])
    # df.insert(0, "startup_name", "Tanso Technologies GmbH")
    # #add a columnn for the startup name as the first column
    # df_shareholder_lists = df_shareholder_lists.append(df)
    # return df_shareholder_lists

# df_shareholder_lists = add_to_df(df_shareholder_lists, openai_json)


UnboundLocalError: cannot access local variable 'df_shareholder_lists' where it is not associated with a value